### **Imports**

In [215]:
import time
start_time = time.time()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import string
import os

### **DataSet** **Upload**

In [216]:
from google.colab import files , drive
drive.mount('/content/gdrive')
dataset = "/content/gdrive/My Drive/Colab Notebooks/covid_data.csv"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### **DataUpload check**

In [217]:
df = pd.read_csv(dataset)

In [218]:
df.shape

(67166, 59)

In [219]:
df.head(5)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498


In [220]:
df.nunique()

iso_code                                   192
continent                                    6
location                                   200
date                                       403
total_cases                              34313
new_cases                                 9391
new_cases_smoothed                       17775
total_deaths                             12928
new_deaths                                2305
new_deaths_smoothed                       5143
total_cases_per_million                  48425
new_cases_per_million                    27581
new_cases_smoothed_per_million           31234
total_deaths_per_million                 28379
new_deaths_per_million                    5548
new_deaths_smoothed_per_million           6782
reproduction_rate                          356
icu_patients                              1938
icu_patients_per_million                  3956
hosp_patients                             3696
hosp_patients_per_million                 6291
weekly_icu_ad

In [221]:
df.dtypes.value_counts()

float64    54
object      5
dtype: int64

In [222]:
df.dtypes

iso_code                                  object
continent                                 object
location                                  object
date                                      object
total_cases                              float64
new_cases                                float64
new_cases_smoothed                       float64
total_deaths                             float64
new_deaths                               float64
new_deaths_smoothed                      float64
total_cases_per_million                  float64
new_cases_per_million                    float64
new_cases_smoothed_per_million           float64
total_deaths_per_million                 float64
new_deaths_per_million                   float64
new_deaths_smoothed_per_million          float64
reproduction_rate                        float64
icu_patients                             float64
icu_patients_per_million                 float64
hosp_patients                            float64
hosp_patients_per_mi

### **Data Preprocessing**

In [223]:
df.drop(['new_cases_smoothed','new_deaths_smoothed','new_cases_smoothed_per_million','new_deaths_smoothed_per_million',
         'new_tests_smoothed_per_thousand','new_tests_smoothed', 'new_vaccinations_smoothed', 'new_vaccinations_smoothed_per_million'], axis=1, inplace=True)

In [224]:
df = df[df.continent == 'Europe'] #filters europe only entries

In [225]:
df.continent.value_counts() #reduced entries from 67K to 16K

Europe    16203
Name: continent, dtype: int64

In [226]:
"""
Loop to select the countries with the most common entries
INPUT: series of countries with number of entries
OUTPUT: list of the most common countries with the number of entries
"""

i = 0
theMostCommonCountry_List = []
while i < 10:
  part_index = str(df.location.value_counts().index[i])
  theMostCommonCountry_List.append( part_index )
  i = i + 1
print(theMostCommonCountry_List)

['France', 'Switzerland', 'Germany', 'Finland', 'Russia', 'United Kingdom', 'Italy', 'Spain', 'Estonia', 'Sweden']


In [227]:
df = df[df.location.isin(theMostCommonCountry_List)]  #filters entries by list of common countries

In [228]:
df = df.reset_index()   #resets index
df.drop(columns=['index','continent'], inplace=True) #drops old index column and continent becasue it is not needed
df

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,EST,Estonia,2020-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,NaN,4.69,78.74,0.871
1,EST,Estonia,2020-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,NaN,4.69,78.74,0.871
2,EST,Estonia,2020-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,NaN,4.69,78.74,0.871
3,EST,Estonia,2020-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,NaN,4.69,78.74,0.871
4,EST,Estonia,2020-02-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,NaN,4.69,78.74,0.871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3742,GBR,United Kingdom,2021-02-02,3863757.0,16906.0,108225.0,1451.0,56915.369,249.035,1594.217,21.374,NaN,3638.0,53.590,31670.0,466.517,NaN,NaN,NaN,NaN,606382.0,71642534.0,1055.336,8.932,0.036,28.1,tests performed,10520433.0,10021471.0,498962.0,376922.0,15.50,14.76,0.73,NaN,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,81.32,0.922
3743,GBR,United Kingdom,2021-02-03,3882972.0,19215.0,109547.0,1322.0,57198.418,283.048,1613.691,19.474,NaN,3625.0,53.398,NaN,NaN,NaN,NaN,NaN,NaN,801949.0,72464146.0,1067.439,11.813,0.034,29.5,tests performed,10992444.0,10490487.0,501957.0,472011.0,16.19,15.45,0.74,NaN,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,81.32,0.922
3744,GBR,United Kingdom,2021-02-04,3903706.0,20734.0,110462.0,915.0,57503.841,305.424,1627.169,13.478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11477040.0,10971047.0,505993.0,484596.0,16.91,16.16,0.75,NaN,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,81.32,0.922
3745,GBR,United Kingdom,2021-02-05,3922910.0,19204.0,111477.0,1015.0,57786.727,282.886,1642.121,14.952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11975267.0,11465210.0,510057.0,498227.0,17.64,16.89,0.75,NaN,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,81.32,0.922


In [229]:
"""
Loop for checking which features are object type
INPUT: pandas series
OUTPUT: list of objects
"""
list_of_objects = []
x=0
while x < len(df.columns):
  if df.dtypes.values[x] == object:
    list_of_objects.append(df.dtypes.index[x])
  x = x + 1
print(list_of_objects)

['iso_code', 'location', 'date', 'tests_units']


In [230]:
df.location.value_counts()

Switzerland       380
France            380
Germany           377
Finland           375
Italy             373
Russia            373
United Kingdom    373
Sweden            372
Estonia           372
Spain             372
Name: location, dtype: int64

In [231]:
correlation_matrix = df.corr().round(2)
correlation_matrix.shape

(46, 46)

In [232]:
#df.corr() #Tried to visualise the matrix

In [309]:
df.isna().sum()

iso_code                                  0
location                                  0
date                                      0
total_cases                              58
new_cases                                58
total_deaths                            383
new_deaths                              383
total_cases_per_million                  58
new_cases_per_million                    58
total_deaths_per_million                383
new_deaths_per_million                  383
reproduction_rate                       521
icu_patients                           1448
icu_patients_per_million               1448
hosp_patients                          1735
hosp_patients_per_million              1735
weekly_icu_admissions                  3562
weekly_icu_admissions_per_million      3562
weekly_hosp_admissions                 3452
weekly_hosp_admissions_per_million     3452
new_tests                              1243
total_tests                            1609
total_tests_per_thousand        

In [313]:
df = df.drop(columns='handwashing_facilities')

KeyError: ignored

In [314]:
df

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index
0,EST,Estonia,2020-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,4.69,78.74,0.871
1,EST,Estonia,2020-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,4.69,78.74,0.871
2,EST,Estonia,2020-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,4.69,78.74,0.871
3,EST,Estonia,2020-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,4.69,78.74,0.871
4,EST,Estonia,2020-02-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1326539.0,31.033,42.7,19.452,13.491,29481.252,0.5,255.569,4.02,24.5,39.3,4.69,78.74,0.871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3742,GBR,United Kingdom,2021-02-02,3863757.0,16906.0,108225.0,1451.0,56915.369,249.035,1594.217,21.374,NaN,3638.0,53.590,31670.0,466.517,NaN,NaN,NaN,NaN,606382.0,71642534.0,1055.336,8.932,0.036,28.1,tests performed,10520433.0,10021471.0,498962.0,376922.0,15.50,14.76,0.73,NaN,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,2.54,81.32,0.922
3743,GBR,United Kingdom,2021-02-03,3882972.0,19215.0,109547.0,1322.0,57198.418,283.048,1613.691,19.474,NaN,3625.0,53.398,NaN,NaN,NaN,NaN,NaN,NaN,801949.0,72464146.0,1067.439,11.813,0.034,29.5,tests performed,10992444.0,10490487.0,501957.0,472011.0,16.19,15.45,0.74,NaN,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,2.54,81.32,0.922
3744,GBR,United Kingdom,2021-02-04,3903706.0,20734.0,110462.0,915.0,57503.841,305.424,1627.169,13.478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11477040.0,10971047.0,505993.0,484596.0,16.91,16.16,0.75,NaN,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,2.54,81.32,0.922
3745,GBR,United Kingdom,2021-02-05,3922910.0,19204.0,111477.0,1015.0,57786.727,282.886,1642.121,14.952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11975267.0,11465210.0,510057.0,498227.0,17.64,16.89,0.75,NaN,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,2.54,81.32,0.922
